# 데이터

In [1]:
import torch
import datasets
import transformers
print(
    datasets.__version__,
transformers.__version__,
torch.__version__)

# 라이브러리 불러오기
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
from sklearn.model_selection import train_test_split

1.14.0 4.11.3 2.3.1+cu118


In [2]:
torch.cuda.empty_cache() 

In [3]:
print(torch.cuda.is_available())
torch._C._cuda_getDeviceCount()

True


1

In [4]:
from datasets import Dataset, load_metric

In [5]:
# 한글 폰트에 문제가 생겼을 때

# 한글 폰트 설치
!apt-get update -qq
!apt-get install -qq fonts-nanum

# 설치한 폰트를 matplotlib에서 사용할 수 있도록 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# 나눔 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 폰트 매니저에 폰트 추가
fm.fontManager.addfont(font_path)
plt.rc('font', family='NanumGothic')  # 폰트 설정

## 데이터 로드

#### 학습 데이터 불러오기

프롬프트로 생성된 '일반 대화' 합성 데이터와 원본 데이터가 합쳐진 파일

In [6]:
train_data_path ="../share/data/conversations.csv"
# 원본 데이터 저장
origin_data = pd.read_csv(train_data_path)

https://huggingface.co/gogamza/kobart-base-v2



In [7]:
# 클래스 별 대화 확인해보기
class_counts = origin_data.groupby('class').size()
class_counts

class
갈취 대화           981
기타 괴롭힘 대화      1094
일반 대화          1000
직장 내 괴롭힘 대화     979
협박 대화           896
dtype: int64

## 전처리

### 기본 전처리

#### 클래스 컬럼 인코딩하기

In [8]:
# 'class'를 'type'으로 매핑하는 딕셔너리 생성하기
class_to_type = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}
type_to_class = {val:key for key, val in class_to_type.items()}

In [9]:
# 'class' 열을 기반으로 새로운 'type' 열 추가하기
origin_data['type'] = origin_data['class'].map(class_to_type)
# 기존 idx, class 컬럼 삭제하기
origin_data.drop(['idx', 'class'], axis=1, inplace=True)
# new_train_data를 train_data에 덮어 씌우기

### 텍스트 전처리

#### 한글 외 문자 삭제
한글, '?', '!', '.', '.', 공백 유지

In [10]:
# 전처리 함수
def preprocess_sentence(sentence): 
    # \n을 공백으로 바꾸기
    sentence = re.sub("\n", " ", sentence)
    
    # (ㄱ-ㅎ, ㅏ-ㅣ, ".", "?", "!", ",", ' ')를 제외한 모든 문자를 없애기
    sentence = re.sub("[^ㄱ-ㅣ가-힣.?!, ]", "", sentence)
    
    # 단어와 구두점(punctuation) 사이에 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    
    return sentence

In [11]:
origin_data['preprocessed'] = origin_data['conversation'].map(preprocess_sentence)
origin_data.drop('conversation', axis=1, inplace=True)

In [12]:
train_data, val_data = train_test_split(origin_data, test_size=0.2, shuffle=True, random_state=42)
val_data, test_data = train_test_split(val_data, test_size=0.5, shuffle=True, random_state=42)

In [13]:
# Convert to Dataset objects
train_ds = Dataset.from_pandas(origin_data, split="train")#.remove_columns(['__index_level_0__'])
val_ds = Dataset.from_pandas(val_data, split="val")#.remove_columns(['__index_level_0__'])
test_ds = Dataset.from_pandas(test_data, split="test")#.remove_columns(['__index_level_0__'])

#### 토큰화 ( 정수 인코딩 )

In [14]:
from transformers import BartTokenizerFast, BartForSequenceClassification

tokenizer = BartTokenizerFast.from_pretrained('hyunwoongko/kobart')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [54]:
def create_input_sequence(sample):
    text = sample["preprocessed"]
    label = sample["type"]
    encoded_sequence = tokenizer(text, truncation = True, padding = True, return_tensors='pt')
    encoded_sequence["labels"] = label
#     encoded_sequence["input_sentence"] = tokenizer.batch_decode(encoded_sequence.input_ids)
    return encoded_sequence


In [55]:
train_dataset = train_ds.map(create_input_sequence, batch_size = 1, remove_columns = ["preprocessed", "type"]).shuffle(seed=42)
valid_dataset = val_ds.map(create_input_sequence, batch_size = 1, remove_columns = ["preprocessed", "type"]).shuffle(seed=42)
test_dataset = test_ds.map(create_input_sequence, batch_size = 1, remove_columns = ["preprocessed", "type"]).shuffle(seed=42)

  0%|          | 0/4950 [00:00<?, ?ex/s]

  0%|          | 0/495 [00:00<?, ?ex/s]

  0%|          | 0/495 [00:00<?, ?ex/s]

In [56]:
print('bos token/id: {} {}\n'
      'eos token/id: {} {}\n'
      'sep_token/id: {} {}\n'
      'additional_special_tokens: {}'.format(
    tokenizer.bos_token, tokenizer.bos_token_id,
    tokenizer.eos_token,tokenizer.eos_token_id,
    tokenizer.sep_token, tokenizer.sep_token_id,
    tokenizer.additional_special_tokens))

bos token/id: <s> 0
eos token/id: </s> 1
sep_token/id: </s> 1
additional_special_tokens: []


In [57]:
from transformers import BartForSequenceClassification, BartConfig
default_model_config = BartConfig.from_pretrained('hyunwoongko/kobart', 
                                          num_labels=5, 
                                          label2id=class_to_type,
                                          id2label=type_to_class,
                                          bos_token_id=tokenizer.bos_token_id,
                                          eos_token_id=tokenizer.eos_token_id,
                                          pad_token_id=tokenizer.pad_token_id,
                                         )
model = BartForSequenceClassification.from_pretrained('hyunwoongko/kobart', config=default_model_config)


loading configuration file https://huggingface.co/hyunwoongko/kobart/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/a06e1ed0b8182dfd1d23c5521c0e5a11ebcd2725cd41ac7f4d099ef7fd6dc12f.db3846b25a7f024766574201ac1e37d3fc3bb9a26f1893cd9aabc80ad2bf4067
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_p

In [58]:
VOCAB_SIZE = tokenizer.vocab_size

In [20]:
word_to_index = tokenizer.vocab

In [21]:

CLS_NUM = 5

## 데이터 분할

In [22]:
from transformers import TrainingArguments, Trainer, EvalPrediction

training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch",
    num_train_epochs = 10,             # Total number of training epochs
    per_device_train_batch_size = 24,  # Batch size per device during training
    per_device_eval_batch_size = 64,   # Batch size for evaluation
    warmup_steps = 500,                # Number of warmup steps for learning rate scheduler
    weight_decay = 0.01,               # Strength of weight decay
)

# training_args = training_args.set_training(
# )

In [23]:
from transformers import EvalPrediction
import numpy as np
# import evaluate
from datasets import load_metric
# metric = evaluate.load("accuracy")
metric_f1 = load_metric("f1")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

def compute_metrics(p: EvalPrediction):
    metric_acc = load_metric("accuracy")
    metric_f1 = load_metric("f1", trust_remote_code=True)
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis = 1)
    result = {}
    result["accuracy"] = metric_acc.compute(predictions = preds, references = p.label_ids)["accuracy"]
    result["f1"] = metric_f1.compute(predictions = preds, references = p.label_ids, average = 'macro')["f1"]
    return result

In [24]:
!pip install wandb==0.16.0 -qq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
%env WANDB_PROJECT=DLthon_finetune_koBart
os.environ['TOKENIZERS_PARALLELISM']='true'
os.environ["WANDB_DISABLED"] = "false"
os.environ['WANDB_API_KEY'] = '746fb761ab2f1b53db2dafef7340caad69224513'
import wandb
from transformers import TrainingArguments, Trainer

# from wandb.keras import WandbCallback
wandb.login()


env: WANDB_PROJECT=DLthon_finetune_koBart


wandb: Currently logged in as: hojae-choi (aiffel_minions). Use `wandb login --relogin` to force relogin


True

In [26]:
transformers.__version__

'4.11.3'

In [37]:
default_training_args = {
    'output_dir':"test_trainer", 
    'evaluation_strategy':"steps",
    'report_to':"wandb",
    'logging_steps':20,
    'load_best_model_at_end':True,
    'num_train_epochs': 10,             # Total number of training epochs
    'per_device_train_batch_size': 24,  # Batch size per device during training
    'per_device_eval_batch_size': 64,   # Batch size for evaluation
    'warmup_steps': 500,                # Number of warmup steps for learning rate scheduler
    'weight_decay': 0.01,               # Strength of weight decay
}

default_model_config = BartConfig.from_pretrained(
    'hyunwoongko/kobart', 
    num_labels=5, 
    label2id=class_to_type,
    id2label=type_to_class,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    )

def separate_config(wandb_config):
    model_config = BartConfig(default_model_config)
    
    training_args = TrainingArguments(
        **default_training_args  
    )
    training_args.num_train_epochs = wandb_config.num_epochs
    
    return model_config, training_args
   

loading configuration file https://huggingface.co/hyunwoongko/kobart/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/a06e1ed0b8182dfd1d23c5521c0e5a11ebcd2725cd41ac7f4d099ef7fd6dc12f.db3846b25a7f024766574201ac1e37d3fc3bb9a26f1893cd9aabc80ad2bf4067
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_p

In [38]:
default_wandb_config = {
#         "vocab" : VOCAB_SIZE,
#         "embeddings" : 128,
#         "units_128" : 128,
#         "units_256" : 256,
#         "units_512" : 512,
#         "units_1024" : 1024,
#         "units_2048" : 2048,
#         "kernel_3" : 3,
#         "kernel_5" : 5,
        "loss" : "sparse_categorical_crossentropy",
        "num_epochs": 10,
    }

In [42]:
def build_model(model_config):
    
    model = BartForSequenceClassification.from_pretrained(
        'hyunwoongko/kobart').to('cuda')
    
    for w in model.model.parameters():
        w._trainable = False
        
    return model

In [52]:
from transformers.integrations import WandbCallback
def train():
    wandb.init(config = default_wandb_config)
    wandb_config = wandb.config
    model_config, training_args = separate_config(wandb_config)
    
    
    model = build_model(model_config)
    
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
    )
    
    trainer.add_callback(WandbCallback())
    
    
    trainer.train()
    wandb.finish()

In [53]:
os.environ["WANDB_LOG_MODEL"] = "checkpoint" 
os.environ["WANDB_PROJECT"] = "DLthon_finetune_koBart"
# torch.cuda.empty_cache() 
train()

using `logging_steps` to initialize `eval_steps` to 20
PyTorch: setting up devices
loading configuration file https://huggingface.co/hyunwoongko/kobart/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/a06e1ed0b8182dfd1d23c5521c0e5a11ebcd2725cd41ac7f4d099ef7fd6dc12f.db3846b25a7f024766574201ac1e37d3fc3bb9a26f1893cd9aabc80ad2bf4067
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [ ]:
%env WANDB_PROJECT=DLthon_finetune_koBart

sweep_config = {
    "name": "sweep_kobart_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.01
            },
        "num_epochs" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

sweep_id = wandb.sweep(sweep_config,
                       entity = 'aiffel_minions',
                       project = 'DLthon_finetune_koBart',
                      )
# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

# 모델링

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login(key = '746fb761ab2f1b53db2dafef7340caad69224513')

In [ ]:
sweep_config = {
    "name": "sweep_test_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

In [ ]:
from  tensorflow import keras
def build_model(config):
    model=keras.models.Sequential()
    model.add(keras.layers.Embedding(config.vocab, config.embeddings))
    model.add(keras.layers.GRU(units = config.units_1, return_sequences = True))
    model.add(keras.layers.GRU(units = config.units_2))
    model.add(keras.layers.Dense(config.units_3, activation='relu'))
    model.add(keras.layers.Dense(config.class_num, activation='softmax'))

    return model
    

In [ ]:
class Config(dict):
    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [ ]:
import tensorflow_addons as tfa
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix, 
    ConfusionMatrixDisplay, 
    classification_report
    )


def train():
    default_config = {
        "vocab" : VOCAB_SIZE,
        "embeddings" : 64,
        "units_1" : 256,
        "units_2" : 256,
        "units_3" : 1024,
        "class_num" : CLS_NUM,
        "learning_rate" : 0.005,
        "optimizer" : "adam",
        "loss" : "sparse_categorical_crossentropy",
        "metrics" : ["accuracy"],
        "epoch" : 10,
        "batch_size" : 32
    }

    wandb.init(config = default_config)
    config = wandb.config
#     config = Config(default_config)

    # Model
    keras.backend.clear_session()
    model = build_model(config)

    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우나 learning rate를 조절할 경우에는 아래와 같은 형태의 코드를 응용합니다.
    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
    
    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    # WandbCallback 함수는 후술합니다.
    
    history = model.fit(X_train, y_train,
              epochs = config.epoch,
              shuffle=True,
              batch_size = config.batch_size,
              validation_data = (X_val, y_val),
              callbacks = [WandbCallback()],
             )
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    pred_test = model.predict(X_test).argmax(axis=1)
    f1_score_res = f1_score(y_test, pred_test, average='micro')
    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.

    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test F1 Score: ": round(f1_score_res * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})
    return history

In [ ]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = 'aiffel_minions',
                       project = 'DLthon_baseline_GRU',
                      )

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

In [ ]:
# train()